### Throughout the exercises for Regression in Python lessons, you will use the following example scenario: 

As a customer analyst, I want to know who has spent the most money with us over their lifetime. 

- I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. 

- I need to do this within an average of $5.00 per customer.

In [1]:
import numpy as np
import pandas as pd
import wrangle

### Q1: Acquire customer_id, monthly_charges, tenure, and total_charges from telco_churn database for all customers with a 2 year contract.

In [4]:
# Inside 'wrangle' is the function to access the database using my env info
# and the function to pull the query from the database ("telco_churn") and
# return it to me as a dataframe

# The 'get_data_from_sql' function returns the data from 'telco_churn' based 
# on the query I have in the function

df = wrangle.get_data_from_sql()
df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.8
2,0016-QLJIS,90.45,65,5957.9
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [6]:
print(df.shape[0]) # number of rows
print(df.shape[1]) # number of columns

1695
4


- So our database has 4 columns with 1,695 rows

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
customer_id        1695 non-null object
monthly_charges    1695 non-null float64
tenure             1695 non-null int64
total_charges      1695 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


 - because it has 1695 non-null objects, customer_id is a unique identifier
 - monthly_charges as floats and tenure as integers makes sense b/c they're numbers
 - but why is total_charges considered a string if it's numbers, too?

In [8]:
df.total_charges.value_counts(dropna=False)

           10
3533.6      2
1161.75     2
1110.05     2
5682.25     2
           ..
6994.6      1
100.35      1
4304.5      1
1851.45     1
3871.85     1
Name: total_charges, Length: 1678, dtype: int64

- looks like there's 10 values that are either spaces or blanks
- find them and decide to either drop them (if unimportant) or adjust

In [9]:
df[df.total_charges == " "] # note space between quotes-  b/c we're looking for a space.

,customer_id,monthly_charges,tenure,total_charges
234,1371-DWPAZ,56.05,0,
416,2520-SGTTA,20.00,0,
453,2775-SEFEE,61.90,0,
505,3115-CZMZD,20.25,0,
524,3213-VVOLG,25.35,0,
678,4075-WKNIU,73.35,0,
716,4367-NUYAO,25.75,0,
726,4472-LVYGI,52.55,0,
941,5709-LVOEQ,80.85,0,
1293,7644-OMVMY,19.85,0,


- tenure is listed as 0, so the total_charges are blank b/c those customers haven't paid
anything yet.  

- If I change their tenure to 1, I am assuming they've been a customer for 
less than a month.  

- This will put a value in my total_charges for one-month's membership.

- Be sure that if I do this, if I am reporting to anybody these findings, make sure I 
am able to outline my reasoning behind why I changed tenure to 1 from 0.

In [11]:
# Replace tenures of 0 with tenures of 1:

df.tenure.replace(0, 1, inplace=True)

In [13]:
# Check to make sure my tenure counts did actually change from 0 to 1:

df.tenure.value_counts().sort_index()

1      12
2       1
3       3
4       4
5       1
     ... 
68     65
69     66
70     88
71    137
72    343
Name: tenure, Length: 72, dtype: int64

In [14]:
df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,
416,2520-SGTTA,20.00,1,
453,2775-SEFEE,61.90,1,
505,3115-CZMZD,20.25,1,
524,3213-VVOLG,25.35,1,
678,4075-WKNIU,73.35,1,
716,4367-NUYAO,25.75,1,
726,4472-LVYGI,52.55,1,
941,5709-LVOEQ,80.85,1,


- Now I gotta do the math so that the monthy_charge and tenure are multiplied to get total_charges

In [15]:
df.total_charges.replace(" ", df.monthly_charges, inplace=True)

In [16]:
df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,56.05
416,2520-SGTTA,20.00,1,20
453,2775-SEFEE,61.90,1,61.9
505,3115-CZMZD,20.25,1,20.25
524,3213-VVOLG,25.35,1,25.35
678,4075-WKNIU,73.35,1,73.35
716,4367-NUYAO,25.75,1,25.75
726,4472-LVYGI,52.55,1,52.55
941,5709-LVOEQ,80.85,1,80.85


In [17]:
# Now we can change all the total_charges from objects to floats

df.total_charges = df.total_charges.astype(float)

In [18]:
# Check to see if all the total_charges are now floats:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
customer_id        1695 non-null object
monthly_charges    1695 non-null float64
tenure             1695 non-null int64
total_charges      1695 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


### Now that I know all this work cleans up the data, I can create the function to do it all for me:

- this will let me easily call the function again and again to do all this work so I don't have to put in all this code

#### VIMP:
- this is the very process I will go through at the start of my projects.  Get the data, clean up the data, and then store the functions doing it in modules I can use throughout the project.

In [21]:
def wrangle_telco():
    df = wrangle.get_data_from_sql()
    df.tenure.replace(0, 1, inplace=True)
    df.total_charges.replace(" ", df.monthly_charges, inplace=True)
    df.total_charges = df.total_charges.astype(float)
    return df

In [22]:
# Now check to see if I can call this function from the wrangle_telco module

wrangle_telco()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75
...,...,...,...,...
1690,9964-WBQDJ,24.40,71,1725.40
1691,9972-EWRJS,19.25,67,1372.90
1692,9975-GPKZU,19.75,46,856.50
1693,9993-LHIEB,67.85,67,4627.65


In [23]:
df = wrangle_telco()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
customer_id        1695 non-null object
monthly_charges    1695 non-null float64
tenure             1695 non-null int64
total_charges      1695 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


### Q3: End with a python file wrangle.py that contains the function, wrangle_telco(), that will acquire the data and return a dataframe cleaned with no missing values.

#### Since my function does what it's supposed to do.  Now it's time to save it in my 'wrangle' file.  Just copy / paste it into 'wrangle' with the other functions and save it.

In [25]:
# Test importing and calling my function to get my cleaned and prepared database:

df = wrangle.wrangle_telco()

AttributeError: module 'wrangle' has no attribute 'wrangle_telco'

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
customer_id        1695 non-null object
monthly_charges    1695 non-null float64
tenure             1695 non-null int64
total_charges      1695 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


In [27]:
df.tenure.value_counts(dropna=False).sort_index()

1      12
2       1
3       3
4       4
5       1
     ... 
68     65
69     66
70     88
71    137
72    343
Name: tenure, Length: 72, dtype: int64

In [28]:
df.total_charges.value_counts(dropna=False).sort_index()

19.85      1
20.00      1
20.25      1
20.35      1
20.45      1
          ..
8547.15    1
8564.75    1
8594.40    1
8670.10    1
8672.45    1
Name: total_charges, Length: 1687, dtype: int64

In [ ]:
# Checks out!